#### 2019/05/19
## ideas
- [ ] 有编号的空间位置(0～1或给定了最大wh)和有序的通道，有了远近概念后，适当的平方层也要有。考虑单层fine tune block
- [ ] 学会使用图像语言来表达异同，对于同类的，就应该把中心的像素的值拿来当做自己的特征之一，觉得相近的，在一定阈值或者或者几个限定之内的前几相似的像素都应该拿来当做自己的特征，以有意义地表达近的概念。这样做主要是因为图像当中似乎没有明确表达坐标的概念，当然也可以反过来搞出一个明确表达坐标的表示法
- [ ] channel之间,或者卷积核之间的施密特正交化
- [ ] 这个xception有点EM的意思, 可以考虑使用EM给他表示, 打印展示, 甚至改进[9.30]
- [ ] 关于xception的神奇疗效, 关于如何在更好处理pooling和stride各自带来的问题,考虑一种在最大,最小,第二大,第二小池上使用4入1出1\*1卷积加权的做法, 都有评价参数,但只有一个参与运算
- [ ] 卷积是怎么评价"这个特征图的这个地方跟那个啥有多像"这个问题的, 猜是卷积核的样子,比如说连续几层的卷积层的filter的组合
- [ ] 使用色彩, erosion dilation, 简化图像信息, 优化颜色位置编码
- [ ] 使用cycle gan生成mask的auto code, 使用drl解释code
- [ ] mask rcnn的结果进行ppo, roialign或rpn之后的特征处开始训练actor critic, 看看最终能不能帮助改善rcnn, 主要是为了提高iou
- [ ] 球坐标经纬度范围加zoom信息及其归一化与置信度，经纬度上的高斯，随时间方差增大
- [ ] region proposal with hsv 记录邻域, 宽阈值筛选， 然后再精细化阈值， 辨认脸型
- [ ] 图像的小片方差也有它的边缘信息, 即, 两个预处理, LAB转换和方差map

## 今日任务
- [ ] 实现腐蚀后膨胀还原


## 以前任务
- [ ] 了解[俄罗斯小哥](https://mp.weixin.qq.com/s/3B3YVGaIRAzEaVojD0Qq3g)的[github](https://github.com/qubvel/segmentation_models.pytorch)
- [ ] 把annotation里面的游程编码一条一条在原图上标出来,搞懂,并且会用coco.py给的接口
- [ ] 搞明白评价接口,能随意使用判定预测结果的准确率
- [x] 开始找DQN pytorch的代码
- [ ] 开始找PPO pytorch的代码
- [ ] 找到rcnn训练崩溃的原因
- [ ] 训练一个神经网络来指导(为instance打分)选择对象以及调整, 考虑bbox大小, 中心位置占画面长宽的比例, 
- [ ] 确定一个默认的overview视角作为后备
- [ ] 设计一个机制,五级打分:没啥变化, 变好了, 变坏了, 太好了, 太坏了, 人只进行这样的评判, 然后可以在线地传递到instance的打分上
- [ ] 结合yolov3与pose body, 实现快速识别
- [ ] 自动提取object的skeleton信息，然后dqn之，以完善边缘的准确率
- [x] 开始探索zoo
- [ ] 赋予AI一些记忆(Memory),哪怕7秒的记忆,让前面一段时间识别的东西有留存,并且指导最新的工作.而不是摄像头的机械记忆.

# 细节
利用erosion dilation缩骨 还原
- ## 色彩区块链，任意选定一个锚点，其他色彩点，可以以lab色彩变换id信息作为索引，记录色彩区块的相对位置，互相引用确定相对位置（考究引用完备性），方便normalize来处理目标在图片中不同大小和旋转的情况，便于分层人类理解
- ## 先根据色彩信息膨胀，提取边缘信息，膨胀细节由监督学习指导，然后腐蚀提取骨架信息，根据骨架画操场椭圆，求二阶导提取边缘关键点，根据操场椭圆曲线进行语义分类，粗分之后带着分类信息再从头run一遍进行细分，如此两三遍锁定概率置信度
- ## 使用meanshift加drl规定出ab圆上的视觉边界，不同粒度关于色彩丰富度的情况
- ## 利用树结构从叶子到根逆推, 我操,平移旋转不变性??? 树的节点包括, (节点id,节点相对坐标, 多对应情况, 节点腐蚀轮数,节点色彩分布(后面再说))
- ## 也可以右侧顺时针旋转搜索关键点
- ## 对于每个关键节点安排一个阈值和置信树, 训练阈值和置信度便能确立模型

# 先形状,后色彩, 阈值信息参与决策
把确定的边界关键点定下来, 不能确定的就不确定, 空在那里, 交由形状匹配器给出置信度,置信模糊的再进一步细化色彩边界再评估
- 阈值信息是指, 提取形状时使用的阈值大小
## 从色彩提取形状要先粗后细,否则会陷入计算量的陷阱

## 粗分形状后,那些被形状隔开的色彩当以平均色和方差数据或直方图数据服务于形状判别

# 考虑使用概率图模型的树来构建置信度传递体系

- ## 完成度描述两件事: 第一, 当前mask占真实mask遗漏了多少, 第二, 当前mask比真实mask多兼并了附近别人的像素有多少

In [7]:
-75.01-960.3+338

-697.31